<a href="https://colab.research.google.com/github/aakriti-dot/Disaster_Classifier/blob/main/disaster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-5a5a467c-fc9f-1a0b-5252-068391c57544)


### Code for all important tasks

In [2]:
import tensorflow as tf

# Create a function to import an image and resize it to be able to be used with our model
def load_and_prep_image(filename, img_shape=224, scale=True):
  """
  Reads in an image from filename, turns it into a tensor and reshapes into
  (224, 224, 3).

  Parameters
  ----------
  filename (str): string filename of target image
  img_shape (int): size to resize target image to, default 224
  scale (bool): whether to scale pixel values to range(0, 1), default True
  """
  # Read in the image
  img = tf.io.read_file(filename)
  # Decode it into a tensor
  img = tf.image.decode_jpeg(img)
  # Resize the image
  img = tf.image.resize(img, [img_shape, img_shape])
  if scale:
    # Rescale the image (get all values between 0 and 1)
    return img/255.
  else:
    return img

# Note: The following confusion matrix code is a remix of Scikit-Learn's
# plot_confusion_matrix function - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.plot_confusion_matrix.html
import itertools
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix

# Our function needs a different name to sklearn's plot_confusion_matrix
def make_confusion_matrix(y_true, y_pred, classes=None, figsize=(10, 10), text_size=15, norm=False, savefig=False):
  """Makes a labelled confusion matrix comparing predictions and ground truth labels.

  If classes is passed, confusion matrix will be labelled, if not, integer class values
  will be used.

  Args:
    y_true: Array of truth labels (must be same shape as y_pred).
    y_pred: Array of predicted labels (must be same shape as y_true).
    classes: Array of class labels (e.g. string form). If `None`, integer labels are used.
    figsize: Size of output figure (default=(10, 10)).
    text_size: Size of output figure text (default=15).
    norm: normalize values or not (default=False).
    savefig: save confusion matrix to file (default=False).

  Returns:
    A labelled confusion matrix plot comparing y_true and y_pred.

  Example usage:
    make_confusion_matrix(y_true=test_labels, # ground truth test labels
                          y_pred=y_preds, # predicted labels
                          classes=class_names, # array of class label names
                          figsize=(15, 15),
                          text_size=10)
  """
  # Create the confustion matrix
  cm = confusion_matrix(y_true, y_pred)
  cm_norm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis] # normalize it
  n_classes = cm.shape[0] # find the number of classes we're dealing with

  # Plot the figure and make it pretty
  fig, ax = plt.subplots(figsize=figsize)
  cax = ax.matshow(cm, cmap=plt.cm.Blues) # colors will represent how 'correct' a class is, darker == better
  fig.colorbar(cax)

  # Are there a list of classes?
  if classes:
    labels = classes
  else:
    labels = np.arange(cm.shape[0])

  # Label the axes
  ax.set(title="Confusion Matrix",
         xlabel="Predicted label",
         ylabel="True label",
         xticks=np.arange(n_classes), # create enough axis slots for each class
         yticks=np.arange(n_classes),
         xticklabels=labels, # axes will labeled with class names (if they exist) or ints
         yticklabels=labels)

  # Make x-axis labels appear on bottom
  ax.xaxis.set_label_position("bottom")
  ax.xaxis.tick_bottom()

  # Set the threshold for different colors
  threshold = (cm.max() + cm.min()) / 2.

  # Plot the text on each cell
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    if norm:
      plt.text(j, i, f"{cm[i, j]} ({cm_norm[i, j]*100:.1f}%)",
              horizontalalignment="center",
              color="white" if cm[i, j] > threshold else "black",
              size=text_size)
    else:
      plt.text(j, i, f"{cm[i, j]}",
              horizontalalignment="center",
              color="white" if cm[i, j] > threshold else "black",
              size=text_size)

  # Save the figure to the current working directory
  if savefig:
    fig.savefig("confusion_matrix.png")

# Make a function to predict on images and plot them (works with multi-class)
def pred_and_plot(model, filename, class_names):
  """
  Imports an image located at filename, makes a prediction on it with
  a trained model and plots the image with the predicted class as the title.
  """
  # Import the target image and preprocess it
  img = load_and_prep_image(filename)

  # Make a prediction
  pred = model.predict(tf.expand_dims(img, axis=0))

  # Get the predicted class
  if len(pred[0]) > 1: # check for multi-class
    pred_class = class_names[pred.argmax()] # if more than one output, take the max
  else:
    pred_class = class_names[int(tf.round(pred)[0][0])] # if only one output, round

  # Plot the image and predicted class
  plt.imshow(img)
  plt.title(f"Prediction: {pred_class}")
  plt.axis(False);

import datetime

def create_tensorboard_callback(dir_name, experiment_name):
  """
  Creates a TensorBoard callback instand to store log files.

  Stores log files with the filepath:
    "dir_name/experiment_name/current_datetime/"

  Args:
    dir_name: target directory to store TensorBoard log files
    experiment_name: name of experiment directory (e.g. efficientnet_model_1)
  """
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=log_dir
  )
  print(f"Saving TensorBoard log files to: {log_dir}")
  return tensorboard_callback

# Plot the validation and training data separately
import matplotlib.pyplot as plt

def plot_loss_curves(history):
  """
  Returns separate loss curves for training and validation metrics.

  Args:
    history: TensorFlow model History object (see: https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/History)
  """
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  accuracy = history.history['accuracy']
  val_accuracy = history.history['val_accuracy']

  epochs = range(len(history.history['loss']))

  # Plot loss
  plt.plot(epochs, loss, label='training_loss')
  plt.plot(epochs, val_loss, label='val_loss')
  plt.title('Loss')
  plt.xlabel('Epochs')
  plt.legend()

  # Plot accuracy
  plt.figure()
  plt.plot(epochs, accuracy, label='training_accuracy')
  plt.plot(epochs, val_accuracy, label='val_accuracy')
  plt.title('Accuracy')
  plt.xlabel('Epochs')
  plt.legend();

def compare_historys(original_history, new_history, initial_epochs=5):
    """
    Compares two TensorFlow model History objects.

    Args:
      original_history: History object from original model (before new_history)
      new_history: History object from continued model training (after original_history)
      initial_epochs: Number of epochs in original_history (new_history plot starts from here)
    """

    # Get original history measurements
    acc = original_history.history["accuracy"]
    loss = original_history.history["loss"]

    val_acc = original_history.history["val_accuracy"]
    val_loss = original_history.history["val_loss"]

    # Combine original history with new history
    total_acc = acc + new_history.history["accuracy"]
    total_loss = loss + new_history.history["loss"]

    total_val_acc = val_acc + new_history.history["val_accuracy"]
    total_val_loss = val_loss + new_history.history["val_loss"]

    # Make plots
    plt.figure(figsize=(8, 8))
    plt.subplot(2, 1, 1)
    plt.plot(total_acc, label='Training Accuracy')
    plt.plot(total_val_acc, label='Validation Accuracy')
    plt.plot([initial_epochs-1, initial_epochs-1],
              plt.ylim(), label='Start Fine Tuning') # reshift plot around epochs
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(2, 1, 2)
    plt.plot(total_loss, label='Training Loss')
    plt.plot(total_val_loss, label='Validation Loss')
    plt.plot([initial_epochs-1, initial_epochs-1],
              plt.ylim(), label='Start Fine Tuning') # reshift plot around epochs
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.xlabel('epoch')
    plt.show()

# Create function to unzip a zipfile into current working directory
# (since we're going to be downloading and unzipping a few files)
import zipfile

def unzip_data(filename):
  """
  Unzips filename into the current working directory.

  Args:
    filename (str): a filepath to a target zip folder to be unzipped.
  """
  zip_ref = zipfile.ZipFile(filename, "r")
  zip_ref.extractall()
  zip_ref.close()

# Walk through an image classification directory and find out how many files (images)
# are in each subdirectory.
import os

def walk_through_dir(dir_path):
  """
  Walks through dir_path returning its contents.

  Args:
    dir_path (str): target directory

  Returns:
    A print out of:
      number of subdiretories in dir_path
      number of images (files) in each subdirectory
      name of each subdirectory
  """
  for dirpath, dirnames, filenames in os.walk(dir_path):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
      y_true: true labels in the form of a 1D array
      y_pred: predicted labels in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall, f1-score.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

### Get the data

In [3]:
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

# Unzip data
unzip_data("nlp_getting_started.zip")

--2024-06-06 17:12:35--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.2.207, 142.250.141.207, 2607:f8b0:4023:c0d::cf, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.2.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2024-06-06 17:12:35 (132 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



### Visualizing the text data

In [4]:
import pandas as pd
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
train_df.head() #1 is disaster, 0 is not

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
# Shuffle training data
train_df_shuffled = train_df.sample(frac=1, random_state=42) # frac for what percent of data (here 1 means 100%)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [6]:
# How many samples of each class?
train_df_shuffled.target.value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [7]:
# visualize some random data
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(disaster)" if target > 0 else "(not disaster)")
  print(f"Text:\n{text}\n")

Target: 0 (not disaster)
Text:
Photo: Failure in structural integrity affects us all. Whether it is a barn raised upon a faulty concrete... http://t.co/cDxE5VMzOj

Target: 0 (not disaster)
Text:
Crying out for more! Set me ablaze

Target: 1 (disaster)
Text:
69 die in Myanmar floods 250000 affected http://t.co/LxjjGyv86A | https://t.co/U9x3perXCO

Target: 1 (disaster)
Text:
Did anyone else see that fireball falling to earth? Look like a plane blew up.

Target: 1 (disaster)
Text:
@Beyonce @NicoleKidman @Oprah these money grubbing kikes need to get a clueI have no money but I can still destroy with telekinesis. Watch.



### Split into train and validation tests

In [8]:
from sklearn.model_selection import train_test_split
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled['text'].to_numpy(),
                                                                            train_df_shuffled['target'].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)

In [9]:
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [10]:
train_sentences[:10]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk',
       '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
       'destroy the free fandom honestly',
       'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
       '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
       'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
      dtype=object)

### Need to convert texts into numbers.
Tokenization vs. Embedding
1. Tokenization - every number will be labelled(direct mapping) from 0 to end(like how one-hot encoding works) but its too time taking and complex for texts.
2. Embedding - Every words will be converted into vectors. Matrix of feature vectors. Depending on size be 128 dimensional vector, 512 vector etc, also gets updated every time neural network keeps learning. Can be created or use pre-learned embedding, like Word2Vec, GloVe etc.

### 1. Text Vectorization(Tokenization)
1. Standardize each sample(lowercase + punctuation stripping)
2. Split each sample into substrings(usually words)
3. recombine substrings into tokens(ngrams)
4. index tokens
5. transform using index into vectors

In [11]:
# Text Vectorization(Tokenization)
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [12]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

#use the default variables
text_vectorizer = TextVectorization(max_tokens=None, #Here None means no cap, that there is no particular number of occurences specified for any word
                                    standardize='lower_and_strip_punctuation',
                                    split='whitespace',
                                    ngrams=None, #treat every token of its own, instead of groups
                                    output_mode='int', #maps token into integer vectors
                                    output_sequence_length=None) #shorter words gets padded to zero, if None according to the max length


In [13]:
#average number of words in the sentences
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [14]:
# Setup custom text vectorization variables
max_vocab_length = 10000 #max number of words to have in the vocabulary
max_length = 15 #max length the sequences will be

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [15]:
#Fit the text vectorizer to the training sentences
text_vectorizer.adapt(train_sentences)

In [16]:
random_sentence = random.choice(train_sentences)
print(f"Original Text:\n{random_sentence}\
      \n\nVectorized version of the text:")
text_vectorizer([random_sentence])

Original Text:
As California fires rage Forest Service sounds the alarm about sharply rising wildfire costs http://t.co/dFYrPpzkPu http://t.co/iwsdbGd1zq      

Vectorized version of the text:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  26,   90,  109, 2765,  188,  386,  919,    2, 1154,   54, 8588,
        2001,  146, 2150,    1]])>

In [17]:
# get the unique words in vocabulary after its fit for tokenizer
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]

print(f"Number of words in vocabulary: {len(words_in_vocab)}")
print(f"Most common words: {top_5_words}")
print(f"Least common words: {bottom_5_words}")

Number of words in vocabulary: 10000
Most common words: ['', '[UNK]', 'the', 'a', 'in']
Least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### 2. Embedding Layer

Tensorflow's predefined embedding layer-
* `input_dim` - size of vocabulary
* `output_dim` - size of output embedding layer as matrix
* `embeddings_initializer` - length of sequences being passed
* `input_length` - length of sequences being passed to embedding layer

In [18]:
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             embeddings_initializer='uniform',
                             input_length=max_length #how long is each input
                             )
embedding

In [19]:
random_sentence = random.choice(train_sentences)

print(f"Original Text:\n{random_sentence}\
      \n\nEmbedded version of the text:\n")

sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original Text:
Boy 11 charged in fatal shooting of 3-year-old http://t.co/FJ7kcRliR7      

Embedded version of the text:



<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.03540293, -0.00545504, -0.01847636, ..., -0.01552851,
          0.00213567, -0.0287885 ],
        [ 0.02276769, -0.01729932,  0.02988758, ..., -0.01148834,
          0.01428651, -0.0216794 ],
        [-0.04866568, -0.00565798,  0.04343904, ...,  0.01741931,
          0.0062946 , -0.03119378],
        ...,
        [-0.01760759,  0.04753568,  0.01805713, ..., -0.04414152,
          0.00397557, -0.00753616],
        [-0.01760759,  0.04753568,  0.01805713, ..., -0.04414152,
          0.00397557, -0.00753616],
        [-0.01760759,  0.04753568,  0.01805713, ..., -0.04414152,
          0.00397557, -0.00753616]]], dtype=float32)>

In [20]:
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.03540293, -0.00545504, -0.01847636, -0.0060186 , -0.00559553,
        -0.0174397 ,  0.02442619, -0.0182712 ,  0.04536042,  0.00281511,
         0.04857409,  0.03127029, -0.00777413,  0.03079834,  0.0475374 ,
        -0.02565495,  0.03192579,  0.04037236, -0.02410758,  0.04892672,
         0.04708514, -0.00154633,  0.00434665,  0.04521877, -0.01742636,
        -0.04032717,  0.04246577,  0.01801195,  0.03164563, -0.04064469,
         0.02128214,  0.01993164,  0.01588699,  0.04755186, -0.02894853,
         0.04380472,  0.00500881,  0.01415814,  0.01515773,  0.02472888,
         0.00977398, -0.00761796, -0.01761653, -0.01484571,  0.03893739,
         0.01970578, -0.00130712, -0.04366796,  0.00542184, -0.02053038,
        -0.0432634 ,  0.03649992,  0.01028186, -0.0118144 , -0.02875257,
         0.0456966 ,  0.00800959,  0.03444742,  0.02451991, -0.02533028,
        -0.02555475, -0.02429407, -0.02726817, -0.00404054,  0.04835453,
  

### Model 0
Baseline model with Naive Bayes with TF-IDF to convert words to numbers

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

#Make pipeline
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()), #converts words to numbers
    ("clf", MultinomialNB()) #model the text
])

#Fit the pipeline on the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [22]:
#Evaluate the model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Baseline Model Accuracy: {baseline_score*100:.2f}%")

Baseline Model Accuracy: 79.27%


In [23]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

Evaluation metrics

In [24]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  model_accuracy = accuracy_score(y_true, y_pred)*100
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

In [25]:
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 1
FNN model - A simple dense model

In [26]:
#Directory to save tensorboard logs
SAVE_DIR = "model_logs"

In [27]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string") #inputs are 1 dimensional strings

x = text_vectorizer(inputs) #Turn the input texts into numbers
x = embedding(x) #create embedding of the numbers

#x = layers.Flatten()(x) #to reshape to the size of labels
x = layers.GlobalAveragePooling1D()(x) #reduce sequence dimension to match the shape of labels

outputs = layers.Dense(1, activation='sigmoid')(x) #sigmoid activation for binary

model_1 = tf.keras.Model(inputs, outputs, name='model_1_dense')


In [28]:
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [29]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [30]:
model_1_history = model_1.fit(train_sentences,
                              train_labels,
                              epochs=10,
                              validation_data =(val_sentences,val_labels),
                              callbacks = [create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                      experiment_name = 'model_1_dense')])

Saving TensorBoard log files to: model_logs/model_1_dense/20240606-171243
Epoch 1/10
215/215 [==============================] - 18s 64ms/step - loss: 0.6119 - accuracy: 0.6960 - val_loss: 0.5360 - val_accuracy: 0.7520
Epoch 2/10
215/215 [==============================] - 2s 10ms/step - loss: 0.4420 - accuracy: 0.8186 - val_loss: 0.4680 - val_accuracy: 0.7835
Epoch 3/10
215/215 [==============================] - 2s 11ms/step - loss: 0.3475 - accuracy: 0.8590 - val_loss: 0.4567 - val_accuracy: 0.7927
Epoch 4/10
215/215 [==============================] - 2s 8ms/step - loss: 0.2854 - accuracy: 0.8917 - val_loss: 0.4650 - val_accuracy: 0.7900
Epoch 5/10
215/215 [==============================] - 1s 6ms/step - loss: 0.2386 - accuracy: 0.9129 - val_loss: 0.4779 - val_accuracy: 0.7861
Epoch 6/10
215/215 [==============================] - 1s 5ms/step - loss: 0.2017 - accuracy: 0.9289 - val_loss: 0.5033 - val_accuracy: 0.7874
Epoch 7/10
215/215 [==============================] - 2s 7ms/step - lo

In [31]:
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.6095 - accuracy: 0.7717


[0.6095061898231506, 0.7716535329818726]

In [32]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

24/24 [==============================] - 0s 2ms/step


(762, 1)

In [33]:
model_1_pred_probs[:10]

array([[0.44259503],
       [0.72466904],
       [0.99940884],
       [0.06521378],
       [0.012533  ],
       [0.97976065],
       [0.8950179 ],
       [0.99972814],
       [0.9980236 ],
       [0.5017642 ]], dtype=float32)

In [34]:
#Convert model prediction probabilities to label format so <0.5 is 0(not disaster) and >0.5 is 1(disaster)
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [35]:
model_1_results = calculate_results(y_true=val_labels,
                                     y_pred=model_1_preds)
model_1_results

{'accuracy': 77.16535433070865,
 'precision': 0.7722289521502119,
 'recall': 0.7716535433070866,
 'f1': 0.7701831305177762}

In [36]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [37]:
# function to compare baseline results to new model results
def compare_baseline_to_new_results(baseline_results, new_model_results):
  for key, value in baseline_results.items():
    print(f"Baseline {key}: {value:.2f}, New {key}: {new_model_results[key]:.2f}, Difference: {new_model_results[key]-value:.2f}")

compare_baseline_to_new_results(baseline_results=baseline_results,
                                new_model_results=model_1_results)

Baseline accuracy: 79.27, New accuracy: 77.17, Difference: -2.10
Baseline precision: 0.81, New precision: 0.77, Difference: -0.04
Baseline recall: 0.79, New recall: 0.77, Difference: -0.02
Baseline f1: 0.79, New f1: 0.77, Difference: -0.02


### Visualize learned embeddings

In [38]:
#Get the vocabulary from the text vectorization layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:5]

(10000, ['', '[UNK]', 'the', 'a', 'in'])

In [39]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [40]:
#Get the weight matrix of the embedding layer
embed_weights = model_1.get_layer('embedding').get_weights()[0]
print(embed_weights.shape)

(10000, 128)


In [41]:
# # # Code below is adapted from: https://www.tensorflow.org/tutorials/text/word_embeddings#retrieve_the_trained_word_embeddings_and_save_them_to_disk
# import io

# # Create output writers
# out_v = io.open("embedding_vectors.tsv", "w", encoding="utf-8")
# out_m = io.open("embedding_metadata.tsv", "w", encoding="utf-8")

# # Write embedding vectors and words to file
# for num, word in enumerate(words_in_vocab):
#   if num == 0:
#      continue # skip padding token
#   vec = embed_weights[num]
#   out_m.write(word + "\n") # write words to file
#   out_v.write("\t".join([str(x) for x in vec]) + "\n") # write corresponding word vector to file
# out_v.close()
# out_m.close()

# # Download files locally to upload to Embedding Projector
# try:
#   from google.colab import files
# except ImportError:
#   pass
# else:
#   files.download("embedding_vectors.tsv")
#   files.download("embedding_metadata.tsv")

### RNNs are useful for sequence data.
Basically, use information from the past to help with the future (this is where the term recurrent comes from). In other words, take an input (X) and compute an output (y) based on all previous inputs.

variants of the RNN:

* Long short-term memory cells (LSTMs).
* Gated recurrent units (GRUs).
* Bidirectional RNN's (passes forward and backward along a sequence, left to right and right to left).

### Model 2: LSTM
Long Short Term Memory
* Input (text) -> Tokenize -> Embedding -> Layers -> Output (label probability)

The main difference will be that we're going to add an LSTM layer between our embedding and output.

And to make sure we're not getting reusing trained embeddings (this would involve data leakage between models, leading to an uneven comparison later on), create another embedding layer (model_2_embedding) for model. The text_vectorizer layer can be reused since it doesn't get updated during training.

In [42]:
# Set random seed and create embedding layer (new embedding layer for each model)
tf.random.set_seed(42)
from tensorflow.keras import layers

model_2_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_2")


# Create LSTM model
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_2_embedding(x)
x = layers.LSTM(64)(x) # return vector for whole sequence
x = layers.Dense(64, activation="relu")(x)

outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

In [43]:
model_2.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [44]:
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_2 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                      

In [45]:
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=10,
                              validation_data= (val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, 'LSTM')])

Saving TensorBoard log files to: model_logs/LSTM/20240606-171329
Epoch 1/10
215/215 [==============================] - 11s 33ms/step - loss: 0.5149 - accuracy: 0.7430 - val_loss: 0.4545 - val_accuracy: 0.7756
Epoch 2/10
215/215 [==============================] - 2s 10ms/step - loss: 0.3189 - accuracy: 0.8702 - val_loss: 0.5097 - val_accuracy: 0.7822
Epoch 3/10
215/215 [==============================] - 3s 14ms/step - loss: 0.2191 - accuracy: 0.9129 - val_loss: 0.5659 - val_accuracy: 0.7690
Epoch 4/10
215/215 [==============================] - 2s 9ms/step - loss: 0.1521 - accuracy: 0.9442 - val_loss: 0.7212 - val_accuracy: 0.7743
Epoch 5/10
215/215 [==============================] - 2s 9ms/step - loss: 0.1064 - accuracy: 0.9575 - val_loss: 0.8034 - val_accuracy: 0.7585
Epoch 6/10
215/215 [==============================] - 2s 7ms/step - loss: 0.0795 - accuracy: 0.9698 - val_loss: 0.9254 - val_accuracy: 0.7743
Epoch 7/10
215/215 [==============================] - 2s 9ms/step - loss: 0.060

In [46]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs.shape, model_2_pred_probs[:10]

24/24 [==============================] - 0s 3ms/step


((762, 1),
 array([[1.7070684e-04],
        [6.2973368e-01],
        [9.9948221e-01],
        [1.6606313e-01],
        [3.9041061e-06],
        [9.9931502e-01],
        [9.5716220e-01],
        [9.9998260e-01],
        [9.9987733e-01],
        [8.7479185e-03]], dtype=float32))

In [47]:
#Convert model prediction probabilities to label format so <0.5 is 0(not disaster) and >0.5 is 1(disaster)
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [48]:
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=model_2_preds)
model_2_results

{'accuracy': 75.45931758530183,
 'precision': 0.7556797828926014,
 'recall': 0.7545931758530183,
 'f1': 0.7524362913232454}

In [49]:
compare_baseline_to_new_results(baseline_results, model_2_results)

Baseline accuracy: 79.27, New accuracy: 75.46, Difference: -3.81
Baseline precision: 0.81, New precision: 0.76, Difference: -0.06
Baseline recall: 0.79, New recall: 0.75, Difference: -0.04
Baseline f1: 0.79, New f1: 0.75, Difference: -0.03


### Model 3: GRU
The GRU cell has similar features to an LSTM cell but has less parameters.

The architecture of the GRU-powered model:

* Input (text) -> Tokenize -> Embedding -> Layers -> Output (label probability)


In [50]:
tf.random.set_seed(42)
from tensorflow.keras import layers

model_3_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_3")

# Build an RNN using the GRU cell
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_3_embedding(x)
x = layers.GRU(64)(x)
x = layers.Dense(64, activation="relu")(x)

outputs = layers.Dense(1, activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")

In [51]:
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [52]:
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_3 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 1)                 65        
                                                       

In [53]:
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=10,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "GRU")])

Saving TensorBoard log files to: model_logs/GRU/20240606-171359
Epoch 1/10
215/215 [==============================] - 10s 34ms/step - loss: 0.5341 - accuracy: 0.7215 - val_loss: 0.4546 - val_accuracy: 0.7756
Epoch 2/10
215/215 [==============================] - 3s 13ms/step - loss: 0.3248 - accuracy: 0.8657 - val_loss: 0.4967 - val_accuracy: 0.7861
Epoch 3/10
215/215 [==============================] - 2s 10ms/step - loss: 0.2217 - accuracy: 0.9158 - val_loss: 0.5621 - val_accuracy: 0.7677
Epoch 4/10
215/215 [==============================] - 2s 9ms/step - loss: 0.1589 - accuracy: 0.9419 - val_loss: 0.6468 - val_accuracy: 0.7795
Epoch 5/10
215/215 [==============================] - 2s 9ms/step - loss: 0.1180 - accuracy: 0.9583 - val_loss: 0.6452 - val_accuracy: 0.7612
Epoch 6/10
215/215 [==============================] - 2s 8ms/step - loss: 0.0891 - accuracy: 0.9692 - val_loss: 0.8942 - val_accuracy: 0.7612
Epoch 7/10
215/215 [==============================] - 2s 9ms/step - loss: 0.0654

In [54]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs.shape, model_3_pred_probs[:10]

24/24 [==============================] - 0s 3ms/step


((762, 1),
 array([[1.2076465e-01],
        [7.1272635e-01],
        [9.9985361e-01],
        [3.5057638e-02],
        [3.5750327e-05],
        [9.9903905e-01],
        [6.4010870e-01],
        [9.9986506e-01],
        [9.9977249e-01],
        [4.3589161e-03]], dtype=float32))

In [55]:
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [56]:
# Calcuate model_3 results
model_3_results = calculate_results(y_true=val_labels,
                                    y_pred=model_3_preds)
model_3_results

{'accuracy': 76.50918635170603,
 'precision': 0.7684966100379833,
 'recall': 0.7650918635170604,
 'f1': 0.7620256878547168}

In [57]:
# Compare to baseline
compare_baseline_to_new_results(baseline_results, model_3_results)

Baseline accuracy: 79.27, New accuracy: 76.51, Difference: -2.76
Baseline precision: 0.81, New precision: 0.77, Difference: -0.04
Baseline recall: 0.79, New recall: 0.77, Difference: -0.03
Baseline f1: 0.79, New f1: 0.76, Difference: -0.02


### Model 4: Bidirectional RNN
A standard RNN will process a sequence from left to right, where as a bidirectional RNN will process the sequence from left to right and then again from right to left.

In practice, many sequence models often see and improvement in performance when using bidirectional RNN's.However, this improvement in performance often comes at the cost of longer training times and increased model parameters (since the model goes left to right and right to left, the number of trainable parameters doubles).

In [66]:
tf.random.set_seed(42)
from tensorflow.keras import layers

model_4_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_4")

# Build a Bidirectional RNN in TensorFlow
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_4_embedding(x)
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x) # bidirectional goes both ways so has double the parameters of a regular LSTM layer
x = layers.Bidirectional(layers.GRU(64))(x)

outputs = layers.Dense(1, activation="sigmoid")(x)

model_4 = tf.keras.Model(inputs, outputs, name="model_4_Bidirectional")

In [67]:
model_4.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [68]:
model_4.summary()

Model: "model_4_Bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_4 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 bidirectional_1 (Bidirecti  (None, 15, 128)           98816     
 onal)                                                           
                                                                 
 bidirectional_2 (Bidirecti  (None, 128)               74496     
 onal)                                                           
                                             

In [69]:
model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs=10,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "bidirectional_RNN")])

Saving TensorBoard log files to: model_logs/bidirectional_RNN/20240606-172238
Epoch 1/10
215/215 [==============================] - 17s 44ms/step - loss: 0.5056 - accuracy: 0.7505 - val_loss: 0.4543 - val_accuracy: 0.7848
Epoch 2/10
215/215 [==============================] - 4s 17ms/step - loss: 0.3111 - accuracy: 0.8723 - val_loss: 0.5110 - val_accuracy: 0.7717
Epoch 3/10
215/215 [==============================] - 3s 15ms/step - loss: 0.2066 - accuracy: 0.9200 - val_loss: 0.5425 - val_accuracy: 0.7625
Epoch 4/10
215/215 [==============================] - 3s 13ms/step - loss: 0.1424 - accuracy: 0.9530 - val_loss: 0.6644 - val_accuracy: 0.7756
Epoch 5/10
215/215 [==============================] - 3s 15ms/step - loss: 0.1055 - accuracy: 0.9656 - val_loss: 0.6534 - val_accuracy: 0.7559
Epoch 6/10
215/215 [==============================] - 3s 15ms/step - loss: 0.0829 - accuracy: 0.9717 - val_loss: 0.7570 - val_accuracy: 0.7546
Epoch 7/10
215/215 [==============================] - 3s 13ms/s

In [70]:
## Make predictions with bidirectional RNN on the validation data
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

24/24 [==============================] - 2s 5ms/step


array([[4.0535677e-02],
       [5.7964945e-01],
       [9.9987268e-01],
       [9.6658605e-01],
       [4.8121405e-05],
       [9.9975818e-01],
       [7.7604473e-01],
       [9.9986327e-01],
       [9.9993265e-01],
       [3.8813204e-02]], dtype=float32)

In [71]:
#Convert prediction probabilities to labels
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 1., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [72]:
#Model results
model_4_results = calculate_results(val_labels, model_4_preds)
model_4_results

{'accuracy': 73.88451443569554,
 'precision': 0.7390392642993366,
 'recall': 0.7388451443569554,
 'f1': 0.7389307354918521}

In [73]:
#Compare to baseline
compare_baseline_to_new_results(baseline_results, model_4_results)

Baseline accuracy: 79.27, New accuracy: 73.88, Difference: -5.38
Baseline precision: 0.81, New precision: 0.74, Difference: -0.07
Baseline recall: 0.79, New recall: 0.74, Difference: -0.05
Baseline f1: 0.79, New f1: 0.74, Difference: -0.05


### Model 5 : Conv1D
Convolutional Neural Network for Text
Have use Conv2D for images, but here text which is 1-Dimensional. Hence will use Conv1D.

A typical CNN architecture for sequences will look:

* Inputs (text) -> Tokenization -> Embedding -> Layers -> Outputs (class probabilities)

The difference again is in the layers component.
* Instead of using an LSTM or GRU cell, we're going to use a `tensorflow.keras.layers.Conv1D()` layer followed by a `tensorflow.keras.layers.GlobablMaxPool1D()` layer.

1-dimensional convolving filters are used as ngram detectors, each filter specializing in a closely-related family of ngrams (an ngram is a collection of n-words, for example, an ngram of 5 might result in "hello, my name is Daniel").

Max-pooling over time extracts the relevant ngrams for making a decision.
The rest of the network classifies the text based on this information.

In [74]:
embedding_test = embedding(text_vectorizer(["this is a test sentence"])) # turn target sentence into embedding

conv_1d = layers.Conv1D(filters=32,
                        kernel_size=5, #looks at 5 words at a time
                        activation="relu") # convolve over target sequence 5 words at a time

conv_1d_output = conv_1d(embedding_test) # pass embedding through 1D convolutional layer

max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output) # get the most important features

embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))